In [18]:
import numpy as np
import pandas as pd

In [19]:
import json
import pydantic


class CeleryConfig(pydantic.BaseModel):
    pass


class SiteConfig(pydantic.BaseModel):
    url: str
    type: str
    routes: list[str]
    confidence: float


class MongoDB(pydantic.BaseModel):
    pass


class Config(pydantic.BaseModel):
    mongodb: MongoDB
    celery: CeleryConfig
    site: list[SiteConfig]


with open("../cluster/configs/celery-beat/config.json", "r") as f:
    sites_cfg = Config.parse_raw(json.dumps(dict(json.load(f))))

sites_cfg_site_routes = {}
for site in sites_cfg.site:
    sites_cfg_site_routes[site.confidence] = set(site.routes)


def valid_or_fake(domain_name: str):
    for confidence, domains in sites_cfg_site_routes.items():
        if domain_name in domains:
            if confidence >= 0.5:
                return True
            else:
                return False

In [20]:
df = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake.gzip")

In [21]:
df["domain"].unique()

array(['monden/', 'it-stiinta/', 'sport/', 'politic/', 'life-death/', '',
       'stiri/economie', 'stiri/externe', 'stiri/actualitate',
       'stiri/actualitate/politica', 'stiri/sport', 'stiri/sci-tech',
       'category/analize/', 'category/news/', 'category/externe/',
       'category/life/', 'category/politica/', 'category/opinii/',
       'category/money/', 'stiri/', 'category/curiozitati-staiti-ca/',
       'category/actualitate/', 'category/stiri-din-romania/',
       'category/mistere-de-pe-mapamond/',
       'category/sanatate-si-medicina/', 'category/stiri-de-pe-mapamond/',
       'category/stirile-zilei/',
       'category/stiri-sanatate-stiri-fitness-stiri-slabire/',
       'category/stiri-internationale/', 'category/timp-liber/',
       'category/stiri-politica/', 'category/utile/', 'covid/'],
      dtype=object)

In [5]:
df

,site,title,domain,content,date
0,https://www.timesnewroman.ro/,Megan Fox e mai bună în dormitor decât în baie,monden/,"Ca să arate că nu este chiar perfectă, Megan F...",2009-10-01 00:00:00+00:00
1,https://www.timesnewroman.ro/,,monden/,"Un coreean a fost arestat pentru că a violat, ...",2009-09-11 00:00:00+00:00
2,https://www.timesnewroman.ro/,Departamentul de scouting de la Radio Zu,monden/,Morar și Buzdugan au împrumutat la Radio Zu mo...,2009-09-18 00:00:00+00:00
3,https://www.timesnewroman.ro/,,monden/,Într-un articol emoționant din ziarul de dumin...,2009-09-14 00:00:00+00:00
4,https://www.timesnewroman.ro/,,monden/,Alina Plugaru mi-a înseninat ziua. Nimic nu cr...,2009-11-01 00:00:00+00:00
...,...,...,...,...,...
78059,https://www.timesnewroman.ro/,"Florina Mihăilă e însărcinată cu oricine, cu t...",monden/,La cît și-o trage nu ar trebui să fie nici o s...,2010-03-08 00:00:00+00:00
78060,https://www.timesnewroman.ro/,Giovanni Becali critică în termeni duri Selecț...,monden/,Nu toată lumea e mulțumită că Paula Seling și ...,2010-03-07 00:00:00+00:00
78061,https://www.timesnewroman.ro/,Sexy Brăileanca o să îmbrace hainele sponsorul...,monden/,Actrița porno Sexy Brăileanca a devenit ieri i...,2010-03-04 00:00:00+00:00
78062,https://www.timesnewroman.ro/,"Femeia ta gonflabilă consumă ""Activia""?",monden/,Ai o femeie gonflabilă în care ai pompat cam m...,2010-03-04 00:00:00+00:00


In [22]:
correspondings = {
    ("it-stiinta/", "covid/", "category/sanatate-si-medicina/", "category/analize/"): {
        "stiri/sci-tech"
    },
    ("sport/", None): {"stiri/sport"},
    ("category/money/", None): {"stiri/economie"},
    ("politic/", "category/politica/", "category/stiri-politica/"): {"stiri/actualitate/politica"},
    ("life-death/", "category/externe/", "category/stiri-internationale/", "category/life/"): {
        "stiri/externe"
    },
    (
        "monden/",
        "category/news/",
        "stiri/",
        "category/stirile-zilei/",
        "category/actualitate/",
    ): {"stiri/actualitate"},
    (
        "category/opinii/",
        "category/curiozitati-staiti-ca/",
        "category/stiri-din-romania/",
        "category/mistere-de-pe-mapamond/",
        "category/stiri-de-pe-mapamond/",
        "category/stiri-sanatate-stiri-fitness-stiri-slabire/",
        "category/timp-liber/",
        "category/utile/",
    ): {"", None},
}


In [23]:
news = []
for fake_domain in correspondings:
    for year in pd.date_range(
        start=min(df["date"]),
        end=max(df["date"]) + pd.Timedelta(days=366),
        freq="y",
        tz="UTC",
    ):
        domains = [
            sum(
                [
                    len(
                        df[
                            (df.date >= year)
                            & (df.date < year + pd.Timedelta(days=366))
                            & (df.domain == domain)
                        ]
                    )
                    for domain in fake_domain
                ]
            )
        ]
        domains.append(
            sum(
                [
                    len(
                        df[
                            (df.date >= year)
                            & (df.date < year + pd.Timedelta(days=366))
                            & (df.domain == domain)
                        ]
                    )
                    for domain in correspondings[fake_domain]
                ]
            )
        )
        print(domains)
        min_length = min(domains)
        news.append({"year": year, "min": min_length})
        print(f"{fake_domain}-{correspondings[fake_domain]} : {min_length}")


[0, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[0, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[2, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[2, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[2, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[77, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[173, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[200, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'} : 0
[211, 0]
('it-stiinta/', 'covid/', 'category/sanatate-si-medicina/', 'category/analize/')-{'stiri/sci-tech'

In [24]:
df_balanced = pd.DataFrame(columns=["_id", "site", "domain", "title", "content", "date"])


In [25]:
df_fake = pd.DataFrame(columns=["_id", "site", "domain", "title", "content", "date"])
df_valid = pd.DataFrame(columns=["_id", "site", "domain", "title", "content", "date"])
for fake_domain in correspondings:
    for domain in fake_domain:
        df_fake = pd.concat(
            [
                df_fake,
                df[(df.domain == domain)],
            ]
        )
    for domain in correspondings[fake_domain]:
        df_valid = pd.concat(
            [
                df_valid,
                df[(df.domain == domain)],
            ]
        )
df_fake = df_fake.sample(frac=1).reset_index(drop=True)
df_valid = df_valid.sample(frac=1).reset_index(drop=True)
for new in news:
    if new["min"] != 0:
        df_balanced = pd.concat(
            [
                df_balanced,
                df_valid[
                    (df_valid.date >= pd.Timestamp(new["year"].to_datetime64(), tz="UTC"))
                    & (
                        df_valid.date
                        < pd.Timestamp(new["year"].to_datetime64(), tz="UTC")
                        + pd.Timedelta(days=366)
                    )
                ][: new["min"]],
                df_fake[
                    (df_fake.date >= pd.Timestamp(new["year"].to_datetime64(), tz="UTC"))
                    & (
                        df_fake.date
                        < pd.Timestamp(new["year"].to_datetime64(), tz="UTC")
                        + pd.Timedelta(days=366)
                    )
                ][: new["min"]],
            ]
        )


In [27]:
df_balanced.to_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")

In [5]:
import json
import pydantic
import numpy as np

class CeleryConfig(pydantic.BaseModel):
    pass


class SiteConfig(pydantic.BaseModel):
    url: str
    type: str
    routes: list[str]
    confidence: float


class MongoDB(pydantic.BaseModel):
    pass


class Config(pydantic.BaseModel):
    mongodb: MongoDB
    celery: CeleryConfig
    site: list[SiteConfig]


with open("../cluster/configs/celery-beat/config.json", "r") as f:
    sites_cfg = Config.parse_raw(json.dumps(dict(json.load(f))))

routes_confidence = {}
for site in sites_cfg.site:
    routes_confidence[site.url] = site.confidence

In [1]:
import pandas as pd
df_balanced = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")


In [7]:
df_balanced["label"] = [routes_confidence[site] for site in df_balanced["site"]]


In [5]:
import numpy as np
from sklearn import preprocessing
label = np.array([label for label in df_balanced["label"]])
df_balanced["label"] = preprocessing.normalize(label.reshape(-1, 1), norm="max")

In [4]:
label = np.array([label for label in df_balanced["label"]])
max_ = max(label)
min_ = min(label)
df_balanced["label"] = (label - min_) / (max_ - min_)

In [6]:
df_balanced.to_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")

In [1]:
import pandas as pd
df_balanced = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")

In [18]:
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
df_balanced = df_balanced[:250]

In [7]:
df_balanced.domain.unique()


array(['stiri/actualitate', 'stiri/externe', 'stiri/actualitate/politica',
       'stiri/economie', 'stiri/sci-tech', 'stiri/sport',
       'category/politica/', 'monden/', 'category/news/',
       'category/externe/', 'politic/', 'category/opinii/', 'life-death/',
       'category/stiri-politica/', 'it-stiinta/',
       'category/stiri-sanatate-stiri-fitness-stiri-slabire/',
       'category/analize/', 'category/stirile-zilei/',
       'category/stiri-internationale/', 'stiri/', 'category/life/',
       'covid/', 'sport/', '', 'category/stiri-de-pe-mapamond/',
       'category/utile/', 'category/timp-liber/', 'category/money/',
       'category/sanatate-si-medicina/',
       'category/mistere-de-pe-mapamond/', 'category/stiri-din-romania/',
       'category/actualitate/', 'category/curiozitati-staiti-ca/'],
      dtype=object)

In [2]:
df_balanced.label.unique()

array([1.        , 0.55882353, 0.        , 0.52941176, 0.67647059,
       0.73529412, 0.29411765])

In [21]:
df_balanced

,_id,site,domain,title,content,date,label
0,None,https://www.digi24.ro/,stiri/actualitate/politica,Conducerea PSD schimbă calendarul de plată a p...,"De luna următoare, dacă vor fi îndeplinite toa...",2018-05-07 14:59:00+00:00,1.000000
1,None,https://www.aktual24.ro/,category/externe/,"Oficial taliban, apel disperat catre Rezistent...",Talibanii doresc sa rezolve situatia din Panjs...,2021-08-24 00:00:00+00:00,0.558824
2,None,https://www.aktual24.ro/,category/politica/,Iata de ce Ponta nu va intra in Parlament. Mes...,"Presedintele Pro Romania, Victor Ponta, a dat ...",2020-10-14 00:00:00+00:00,0.558824
3,None,https://www.digi24.ro/,stiri/actualitate/politica,Președintele Senatului: Aici nu este fabrică d...,"""Arestarea unei persoane, a unui membru al Par...",2015-03-16 15:02:00+00:00,1.000000
4,None,https://www.digi24.ro/,stiri/actualitate,Suspiciuni că polițiștii au ratat 30 de saci c...,"Șeful Poliției Române, chestorul Ioan Buda, a ...",2019-07-15 11:18:00+00:00,1.000000
...,...,...,...,...,...,...,...
245,None,https://www.digi24.ro/,stiri/actualitate,Cazul fetelor dispărute la Caracal: rămășițe u...,Suspectul de 66 de ani în cazul fetelor dispăr...,2019-07-26 13:31:00+00:00,1.000000
246,None,https://www.activenews.ro/,stiri/,Părinți sfâșiați de durere acuză guvernul că a...,Numeroși părinți spun că au fost înșelați de g...,2021-11-18 00:00:00+00:00,0.676471
247,None,https://www.aktual24.ro/,category/externe/,Viktor Orban s-a oferit sa le construiasca si ...,Premierul ungar Viktor Orban a sugerat marti c...,2018-07-25 00:00:00+00:00,0.558824
248,None,https://www.digi24.ro/,stiri/actualitate,Interpol: Crima organizată va încerca să vândă...,Organizația internațională Interpol a transmis...,2020-12-02 19:13:00+00:00,1.000000


In [22]:
df_balanced.to_parquet("../data/processed/toy_dataset.gzip")

In [53]:
import re
import pickle

import spacy
import numpy as np
import pandas as pd

from src.cleaner.model import Cleaner
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected"

In [1]:
import pandas as pd
df_balanced = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")

In [2]:
df_balanced

,_id,site,domain,title,content,date,label
8,None,https://www.digi24.ro/,stiri/externe,"Bursele asiatice în declin, petrolul în cădere...",Bursele asiatice înregistrau scăderi pe linie ...,2020-03-12 09:31:00+00:00,1.000000
9,None,https://www.digi24.ro/,stiri/externe,Șase turiști faliți au fost salvați după 25 de...,Șase turiști care locuiau într-o peșteră din I...,2020-04-21 11:54:00+00:00,1.000000
14,None,https://www.digi24.ro/,stiri/externe,"Germania închide magazine ""neesențiale"" și bor...",Germania a decis luni închiderea magazinelor '...,2020-03-16 19:24:00+00:00,1.000000
18,None,https://www.digi24.ro/,stiri/externe,Galerie Foto Imagini cutremurătoare cu o tânăr...,"O tânără îngrozită a fost târâtă, luni, pe str...",2020-03-02 15:12:00+00:00,1.000000
22,None,https://www.digi24.ro/,stiri/externe,Încă o țară europeană închide barurile și rest...,"Olanda va fi supusă ""izolării parțiale"" de mie...",2020-10-13 22:25:00+00:00,1.000000
...,...,...,...,...,...,...,...
1128,None,https://www.timesnewroman.ro/,it-stiinta/,Antena 3 a dat imagini din Starcraft și susțin...,Postul de televiziune Antena 3 a difuzat în ac...,2022-03-01 00:00:00+00:00,0.000000
1129,None,https://www.aktual24.ro/,category/externe/,Negocierile s-au incheiat. Ce au stabilit dele...,Negocierile de pace de la granita cu Belarus s...,2022-02-28 00:00:00+00:00,0.558824
1144,None,https://www.aktual24.ro/,category/externe/,"Urale, inca un avion al Rusiei a fost doborat ...",Unitatile ucrainene de aparare antiaeriana au ...,2022-03-06 00:00:00+00:00,0.558824
1227,None,https://www.aktual24.ro/,category/externe/,Premiera in istoria Rusiei. Moscova anunta ca ...,Fortele militare din cadrul Organizatiei Trata...,2022-01-13 00:00:00+00:00,0.558824


In [8]:
import numpy as np
from sklearn import preprocessing
label = np.array([label for label in df_balanced["label"]])
scaller = preprocessing.MinMaxScaler()
label = scaller.fit_transform(label.reshape(-1, 1))

df_balanced["label"] = (label.reshape(-1, 1))

In [9]:
import pickle
with open("scaller.pkl", "wb") as f:
    pickle.dump(scaller, f)

In [1]:
scaller.inverse_transform(np.array([0.74576271]).reshape(-1, 1))

NameError: name 'scaller' is not defined

In [10]:
df_balanced.label.unique()

array([1.        , 0.74576271, 0.        , 0.72881356, 0.81355932,
       0.84745763, 0.59322034])

In [52]:
domain_number

[{'label': 1.0, 'length': 47833, 'type': 'valid'},
 {'label': 0.5588235294117647, 'length': 34569, 'type': 'valid'},
 {'label': 0.0, 'length': 7647, 'type': 'fake'},
 {'label': 0.5294117647058825, 'length': 3385, 'type': 'valid'},
 {'label': 0.6764705882352942, 'length': 2879, 'type': 'valid'},
 {'label': 0.7352941176470589, 'length': 674, 'type': 'valid'},
 {'label': 0.2941176470588236, 'length': 27, 'type': 'fake'}]

In [60]:
domain_number = []

for label in df_balanced.label.unique():
    domain_number.append(
        {
            "label": label,
            "length": len(df_balanced[df_balanced.label == label]),
            "type": "valid" if label > 0.5 else "fake",
        }
    )


fig = px.histogram(
    domain_number, x="length", y="type", color="type", text_auto=".2s", barmode="group"
)
fig.show()


In [10]:
import pandas as pd
df_small = pd.DataFrame(columns=df_balanced.columns)
for label in df_balanced.label.unique():
    for _ in range(100):
        df_small = pd.concat(
            [
                df_small,
                df_balanced[df_balanced.label == label].sample(1).reset_index(drop=True),
            ]
        )


In [4]:
df_small.to_parquet("../data/processed/toy_dataset.gzip")

,_id,site,domain,title,content,date,label
0,None,https://www.digi24.ro/,stiri/actualitate,Camera Deputaților anunță că va cumpăra 15 aut...,"Camera Deputaților vrea să cumpere, prin licit...",2019-09-29 16:45:00+00:00,1.000000
0,None,https://www.digi24.ro/,stiri/economie,Shopping City Sibiu și-a schimbat proprietarul,Foto: Gulliver/Getty ImagesTranzacția a inclus...,2016-07-11 14:19:00+00:00,1.000000
0,None,https://www.digi24.ro/,stiri/economie,La Cernobîl se va produce din nou energie,"La trei decenii de la explozia de la Cernobîl,...",2018-04-29 19:18:00+00:00,1.000000
0,None,https://www.digi24.ro/,stiri/actualitate/politica,"Iohannis, mesaj cu ocazia Școlii de Vară a TNL...","Președintele României, Klaus Iohannis, a trans...",2021-09-18 19:44:00+00:00,1.000000
0,None,https://www.digi24.ro/,stiri/externe,"""Marea lege a coronavirusului"". Guvernul israe...",Deputații israelieni au votat miercuri în a do...,2020-07-23 05:33:00+00:00,1.000000
...,...,...,...,...,...,...,...
0,None,https://infoalert.ro/,category/curiozitati-staiti-ca/,7 reguli care trebuie urmate pentru a duce o v...,,2022-02-18 00:00:00+00:00,0.294118
0,None,https://infoalert.ro/,category/mistere-de-pe-mapamond/,"Triunghiul Bermudelor, într-o pădure celebră d...","Pădurea Hoia-Baciu, renumită pentru copacii să...",2022-03-02 00:00:00+00:00,0.294118
0,None,https://infoalert.ro/,category/stiri-de-pe-mapamond/,"Previziuni Carmen Hara: ""Rusia va ataca. Situa...","Carmen Harra a vorbit, la Antena 3, despre un ...",2022-02-08 00:00:00+00:00,0.294118
0,None,https://infoalert.ro/,category/stiri-din-romania/,"Românii între 20 si 35 de ani, în caz de războ...",Cine poate fi chemat obligatoriu în Armată în ...,2022-02-06 00:00:00+00:00,0.294118


In [7]:
import pandas as pd
df_balanced = pd.read_parquet("../data/processed/scrapped_cleaned_valid_and_fake_balanced.gzip")